# Step 03
# Predict segmentations

In [28]:
%matplotlib inline

In [29]:
import os
import os.path

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.morphology

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder

# Configuration

In [24]:
from config import config_vars

# Partition of the data to make predictions (test or validation)
partition = "validation"

experiment_name = '09'

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'augment_images': True,
 'batch_size': 10,
 'boundary_boost_factor': 1,
 'boundary_labels_dir': '/storage/data/2018_tim_tracking/unet/boundary_labels/',
 'boundary_size': 2,
 'cell_min_size': 40,
 'create_split_files': True,
 'crop_size': 256,
 'csv_log_file': '/storage/data/2018_tim_tracking/unet/experiments/08/log.csv',
 'elastic_augmentations': 10,
 'elastic_distortion': 5,
 'elastic_points': 16,
 'epochs': 15,
 'experiment_dir': '/storage/data/2018_tim_tracking/unet/experiments/08/out/',
 'labels_out_dir': '/storage/data/2018_tim_tracking/unet/experiments/08/out/segm/',
 'learning_rate': 1e-05,
 'max_training_images': 0,
 'min_nucleus_size': 40,
 'model_file': '/storage/data/2018_tim_tracking/unet/experiments/08/model.hdf5',
 'normalized_images_dir': '/storage/data/2018_tim_tracking/unet/norm_images/',
 'object_dilation': 3,
 'path_files_test': '/storage/data/2018_tim_tracking/unet/test.txt',
 'path_files_training': '/storage/data/2018_tim_tracking/unet/training.txt',
 'path_files

In [25]:
config_vars["boundary_boost_factor"]

1

In [26]:
# Device configuration

# Use the following configuration if you want to test on CPUs
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
# configuration = tf.ConfigProto(
#       intra_op_parallelism_threads=1,
#       inter_op_parallelism_threads=1)

# Configuration to run on GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"

session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

# Load images and run predictions

In [27]:
#image_names = [f for f in data_partitions[partition] if f.startswith("IXM")]
image_names = [os.path.join(config_vars["normalized_images_dir"], f) for f in data_partitions[partition]]

imagebuffer = skimage.io.imread_collection(image_names)

images = imagebuffer.concatenate()

dim1 = images.shape[1]
dim2 = images.shape[2]

images = images.reshape((-1, dim1, dim2, 1))

# preprocess (assuming images are encoded as 8-bits in the preprocessing step)
images = images / 255

# build model and load weights
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.load_weights(config_vars["model_file"])

# Normal prediction time
predictions = model.predict(images, batch_size=1)

model.summary()

FileNotFoundError: [Errno 2] No such file or directory: '/storage/data/2018_tim_tracking/unet/norm_images/training_set_058_f58_s4_z3.png'

In [ ]:
config_vars["cell_min_size"] = 10
config_vars["boundary_boost_factor"] = 0.5

# Transform predictions to label matrices

In [ ]:
for i in range(len(images)):

    filename = imagebuffer.files[i]
    filename = os.path.basename(filename)
    print(filename)
    
    probmap = predictions[i].squeeze()
    
    plt.imshow(probmap)
    plt.show()
    
    skimage.io.imsave(config_vars["probmap_out_dir"] + filename, probmap)
    
    pred = utils.metrics.probmap_to_pred(probmap, config_vars["boundary_boost_factor"], 1.5)

    plt.imshow(pred)
    plt.show()
    
    label = utils.metrics.pred_to_label(pred, config_vars["cell_min_size"])
    
    plt.imshow(label)
    plt.show()
    
    skimage.io.imsave(config_vars["labels_out_dir"] + filename, label)